In [151]:
import os
import requests
import xlrd
import pandas as pd
from dotenv import load_dotenv
import numpy as np
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [152]:
api_key = os.getenv("KEY")
url = f"https://api.eia.gov/v2/petroleum/pri/spt/data/?frequency=daily&data[0]=value&facets[product][]=EPCWTI&start=1986-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key}"

In [153]:
r = requests.get(url)

In [154]:
json = r.json()

In [155]:
df = pd.DataFrame(json['response']['data'])
df['period'] = pd.to_datetime(df['period'], infer_datetime_format=True)
df['year'] = pd.DatetimeIndex(df['period']).year
df['month'] = pd.DatetimeIndex(df['period']).month
df['day'] = pd.DatetimeIndex(df['period']).day
df = df.drop(columns=['product', 'period', 'product-name', 'process', 'duoarea', 'area-name', 'units', 'series-description', 'series', 'process-name'])
df

,value,year,month,day
0,79.86,2023,1,24
1,81.62,2023,1,23
2,81.27,2023,1,20
3,80.31,2023,1,19
4,79.53,2023,1,18
...,...,...,...,...
4995,29.48,2003,4,1
4996,31.14,2003,3,31
4997,30.21,2003,3,28
4998,30.31,2003,3,27


In [172]:
test = df.drop(df.tail(3900).index, inplace = False)
print(test)
train = df
train_y = train['value']
train_X = train.drop(columns='value')
test_y = test['value']
test_X = test.drop(columns='value')

     value  year  month  day
0    79.86  2023      1   24
1    81.62  2023      1   23
2    81.27  2023      1   20
3    80.31  2023      1   19
4    79.53  2023      1   18
..     ...   ...    ...  ...
365  63.69  2021      8   19
366  65.36  2021      8   18
367  66.50  2021      8   17
368  67.44  2021      8   16
369  68.36  2021      8   13

[370 rows x 4 columns]


In [171]:
# Splitting

In [173]:
# Instantiation
xgb_r = xg.XGBRegressor(objective='reg:linear', n_estimators=10, seed=123)

In [174]:
# Fitting the model
xgb_r.fit(train_X, train_y)

[15:27:53] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [175]:
# Predict the model
pred = xgb_r.predict(test_X)

In [197]:
num = 40
print(df.iloc[[num]])
print('guess: ', pred[num], '\n', test_X.iloc[[num]])

    value  year  month  day
40  77.93  2022     11   23
guess:  80.77163 
     year  month  day
40  2022     11   23


In [180]:
# RMSE Computation
rmse = np.sqrt(MSE(test_y, pred))
print("RMSE : % f" %(rmse))

RMSE :  4.492505
